# Face PCA

This code is to generate the Face PCA as well as to export such database.
The workflow follows as this:

- Open the necessary files, scripts, and data
- Remove the IDs than don't have Covariates or 3D face data
- PCA workflow:
    - Initial PCA taking a random sample of the same number of males and females, detect outliers using the mahalanobis distance with respect to the origin, and remove them
    - New PCA using the non-outliers, and taking random sample of the same number of males and females
    - To detect possible differences due to sampling errors, I will sample a number of *n* times and measure the correlation of the PC scores
    - If correlation is high enough, take the first result of the PCA
- Export final dataset

# Preliminaries

Locate the corresponding folder and load the database

In [1]:
%Folders
folder.codePath = 'C:\Users\tzarzar\Box Sync\Research\GeneralCode\Matlab Code';
addpath(genpath(folder.codePath));
folder.dataPath16 = 'R:\ShriverLab\Facial features input files and databases\PSU_KU_WFS2016';
folder.databases  = 'C:\Users\tzarzar\Box Sync\Research\FacialSD\DataBases';
folder.results    = 'C:\Users\tzarzar\Box Sync\Research\FacialSD\Results\FacePCA';
cd(folder.dataPath16);
load PENNDATA_AccumulatedShapeData; %loading faces (Data)
load RefScan; %loading AM (RefScan)
load NewMaskIndex; %loading AM trimming index (MaskIndex)
crop(RefScan, 'VertexIndex', MaskIndex); %Trimming the RefScan
cd(folder.databases);
Covariates = readtable('Covariates.csv');

Get the intersection between Covariates and face data

In [2]:
[keep1, keep2] = GetIntersection(Data, Covariates);
Data           = reduceData(Data, keep1);
Covariates     = Covariates(keep2, :);
nFaces         = length(Data.Names)
nCovariates    = length(Covariates.ID)


nFaces =

        5939


nCovariates =

        5939



Get the number of males and females in the dataset

In [3]:
sum(strcmp(Covariates.Sex, 'Female'))
sum(strcmp(Covariates.Sex, 'Male'))


ans =

        3617


ans =

        2284



## PCA Analysis and export

Generalized Procrustes Analysis

In [4]:
TotalShape = [Data.Shape, Data.NormShape]; %Concatenation of original faces (Data.Shape) and their reflection (Data.Norm)
model      = shapePCA; %Creating an empty shape space object
model.RefScan = clone(RefScan); %Defining the AM that was used to create the shape space
AlignedData   = LSGenProcrustes(model, TotalShape, true, 3, RefScan);

Starting parallel pool (parpool) using the 'local' profile ...
connected to 4 workers.


Decomposing faces into components of symmetry and asymmetry

In [5]:
OrigHead = AlignedData(:, 1:nFaces);
ReflHead = AlignedData(:, nFaces+1:end);
SymHead  = (OrigHead + ReflHead)/2; %facial symmetry component
AsymHead = (OrigHead - ReflHead) + mean(SymHead, 2); %facial asymmetry component

In [6]:
size(SymHead)


ans =

       20370        5939



Running the Face PCA. I will use a random sample of 2000 males and females

In [38]:
[sample_fem, ind1]  = datasample(SymHead(:,strcmp(Covariates.Sex, 'Female')), 2000, 2);

[sample_male, ind2] = datasample(SymHead(:,strcmp(Covariates.Sex, 'Male')), 2000, 2);
total_sample = [sample_fem sample_male];

In [67]:
vect = [1:size(SymHead(:,strcmp(Covariates.Sex, 'Female')), 2)];
sum(~ismember(vect, ind1))


ans =

        2081



Matrix dimensions must agree.



In [46]:
SymHead(:,strcmp(Covariates.Sex, 'Female'))


In [29]:
getAverage(model, total_sample); %Compute the average head
getModel(model, total_sample); 
means = mean(total_sample, 2);
stripPercVar(model, 98);
model


model = 

  shapePCA with properties:

    AvgVertices: [3×6790 double]
         AvgVec: [20370×1 double]
        RefScan: [1×1 meshObj]
        Average: [1×1 meshObj]
         EigVal: [82×1 double]
         EigVec: [20370×82 double]
         Tcoeff: [4000×82 double]
       AvgCoeff: [82×1 double]
      Centering: 1
           nrEV: 82
         EigStd: [82×1 double]
      Explained: [82×1 double]
              n: 4000
              U: [4000×82 double]
              S: [82×1 double]
              V: [20370×82 double]
           Type: 'shapePCA'



In [35]:
test = getModel(model, sample_fem);
test


test = 

  shapePCA with properties:

    AvgVertices: [3×6790 double]
         AvgVec: [20370×1 double]
        RefScan: [1×1 meshObj]
        Average: [1×1 meshObj]
         EigVal: [1999×1 double]
         EigVec: [20370×1999 double]
         Tcoeff: [2000×1999 double]
       AvgCoeff: [1999×1 double]
      Centering: 1
           nrEV: 1999
         EigStd: [1999×1 double]
      Explained: [1999×1 double]
              n: 2000
              U: [2000×1999 double]
              S: [1999×1 double]
              V: [20370×1999 double]
           Type: 'shapePCA'



In [37]:
model.Tcoeff(1:10, 1:10)
test.Tcoeff(1:10, 1:10)


ans =

   -0.9523   -0.2297   -0.5964   -0.8118    0.9104    0.7378   -0.4918    0.8523   -0.2297    0.4124
   -1.8794    1.3152    0.6514   -0.4180    1.7252   -0.2966    0.7007    0.5035   -0.3870    0.7176
   -0.8297   -2.1154    0.6421    0.9738    1.3152    0.4682    0.0947    0.3457   -0.0501    0.0312
    4.8613   -0.2636    1.2242    1.8354   -1.9837   -1.1129   -0.0856    0.1210    0.9757    0.1344
   -1.1315    0.3352    0.4625    0.6495    1.2035    0.5445   -0.2102    0.2990   -0.1373    0.6338
    2.8213    0.4656    1.9864   -0.1772    0.0302   -0.6697   -0.3201   -0.0224   -0.5344    0.1551
   -0.4231   -0.8728   -1.7936   -0.8094    0.5273    1.0157   -0.9873    0.1954   -0.0399    0.4285
    1.3004   -1.1405    2.1615   -1.0495   -0.2186   -0.3949   -1.0823   -0.3709   -0.4467    0.5035
    0.7745   -0.8497    0.2355    1.1825   -0.1823    1.1772    0.6210   -0.1214   -0.1099   -0.5191
    2.2759   -0.7961   -1.1174    1.4912    1.4782   -0.6309   -0.5524    0.1935   

# Removing outliers

We will use the mahalanobis distance with respect to the origin as a measure of similarity

In [30]:
origin    = zeros(1, size(model.Tcoeff, 2)); 
mahaldist = sqrt(sum(( (model.Tcoeff ./ model.EigStd') - origin ) .^ 2, 2 ));
size(mahaldist)


ans =

        4000           1



We will define outliers if they are 3 scaled median absolute deviation (MAD) away from the median of the mahaldist distribution.
Below you can see the individual IDs identified as outliers

In [31]:
sum(isoutlier(mahaldist))
Covariates.ID(isoutlier(mahaldist))


ans =

   164


ans =

  164×1 cell array

    '13012'
    '131038'
    '131200'
    '131222'
    '132033'
    '132096'
    '140050'
    '140061'
    '140097'
    '140098'
    '140117'
    '140243'
    '140247'
    '140340'
    '140401'
    '140408'
    '140411'
    '140436'
    '140447'
    '140481'
    '140498'
    '140516'
    '140537'
    '140575'
    '140610'
    '140629'
    '140653'
    '140701'
    '140758'
    '140773'
    '140794'
    '140805'
    '140809'
    '140816'
    '140817'
    '140876'
    '140924'
    '140954'
    '140991'
    '140994'
    '141005'
    '141015'
    '141102'
    '141107'
    '141236'
    '141247'
    '141294'
    '141296'
    '141309'
    '141312'
    '141358'
    '141418'
    '141421'
    '141430'
    '141499'
    '141522'
    '141573'
    '141592'
    '141596'
    '141604'
    '141638'
    '141737'
    '141742'
    '141772'
    '141820'
    '141864'
    '141884'
    '141913'
    '141985'
    '141992'
    '143016'
    '143023'
    '143056'
    '143

# Creating database for export

Exporting database with the identified outliers removed. 
Also, we will compute BMI

In [19]:
PCnames        = strseq('PC', 1:model.nrEV);
Covariates.BMI = Covariates.Weight ./ ( (Covariates.Height ./ 100) .^2);
coeffs         = [Covariates, array2table(model.Tcoeff, 'VariableNames', PCnames)];
cd(folder.results)
csvwrite('eigenvalues.csv', model.EigVal);
csvwrite('eigenvectors.csv', model.EigVec);
csvwrite('means.csv', means);
csvwrite('facets.csv', model.Average.Faces');
writetable(coeffs(~isoutlier(mahaldist), :), 'coeffs.csv')